In [ ]:
!pip install -q tensorflow tensorflow-datasets

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam


In [ ]:
# Load IMDb dataset (as_supervised=True gives (text, label) pairs)
train_data, test_data = tfds.load("imdb_reviews", split=["train", "test"], as_supervised=True)

# Extract raw text and labels from the dataset
train_texts = []
train_labels = []
test_texts = []
test_labels = []

for text, label in train_data:
    train_texts.append(text.numpy().decode("utf-8"))
    train_labels.append(label.numpy())

for text, label in test_data:
    test_texts.append(text.numpy().decode("utf-8"))
    test_labels.append(label.numpy())


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.ECW949_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.ECW949_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.ECW949_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the tokenizer and fit on training texts
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Pad the sequences to the same length
x_train = pad_sequences(train_sequences, maxlen=200)
x_test = pad_sequences(test_sequences, maxlen=200)

# Convert labels to arrays
y_train = np.array(train_labels)
y_test = np.array(test_labels)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=200),  # specify input_length
    LSTM(64),
    Dense(1, activation='sigmoid')
])

model.build(input_shape=(None, 200))  # Batch size unspecified, sequence length 200
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 200, 64)             │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 673,089 (2.57 MB)

 Trainable params: 673,089 (2.57 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 200, 64)             │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 673,089 (2.57 MB)

 Trainable params: 673,089 (2.57 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    x_train,
    y_train,
    epochs=5,               # You can increase this for better accuracy
    batch_size=32,
    validation_data=(x_test, y_test)
)


Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 105s 134ms/step - accuracy: 0.9510 - loss: 0.1370 - val_accuracy: 0.8663 - val_loss: 0.3618
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 161s 159ms/step - accuracy: 0.9671 - loss: 0.0988 - val_accuracy: 0.8520 - val_loss: 0.3927
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 123s 134ms/step - accuracy: 0.9737 - loss: 0.0765 - val_accuracy: 0.8579 - val_loss: 0.4822
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 141s 133ms/step - accuracy: 0.9657 - loss: 0.0950 - val_accuracy: 0.8420 - val_loss: 0.4851
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 124s 159ms/step - accuracy: 0.9708 - loss: 0.0867 - val_accuracy: 0.8424 - val_loss: 0.5415


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"\nTest Accuracy: {accuracy:.4f}")

# Function to predict sentiment from raw text
def predict_sentiment(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(padded)
    label = "Positive" if prediction[0][0] > 0.5 else "Negative"
    print(f"Review: {text}")
    print(f"Predicted Sentiment: {label} ")

# Example usage

predict_sentiment("It was boring and too long.")


782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 30ms/step - accuracy: 0.8426 - loss: 0.5469

Test Accuracy: 0.8424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Review: It was boring and too long.
Predicted Sentiment: Negative 
